In [ ]:
import copy
import random

from nbmetalog import nbmetalog as nbm
import pandas as pd
import seaborn as sns

import pylib

random.seed(1)


In [ ]:
nbm.print_metadata()


In [ ]:
def get_deposition_site_at_rank(rank: int) -> int:
    return pylib.hanoi.get_hanoi_value_at_index(rank)


In [ ]:
def get_deposition_rank_at_site(site: int, num_site_depositions: int) -> int:
    if num_site_depositions == 0:
        return 0

    num_incidences = pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
        site,
        num_site_depositions - 1,
    )
    if num_incidences == 0:
        return 0

    return pylib.hanoi.get_index_of_hanoi_value_nth_incidence(site, num_incidences - 1)


In [ ]:
surface_size = 64
num_generations = 1000

surface_timestamps = [0] * surface_size
surface_values = [random.choice([0, 1]) for __ in range(surface_size)]


In [ ]:
surface_timestamps_records = []
for generation in range(num_generations):
    target_site = get_deposition_site_at_rank(generation)
    surface_values[target_site] = random.choice([0, 1])
    surface_timestamps[target_site] = generation

    assert [
        get_deposition_rank_at_site(site, generation + 1)
        for site in range(surface_size)
    ] == surface_timestamps

    for site, timestamp in enumerate(surface_timestamps):
        surface_timestamps_records.append(
            {
                "generation": generation,
                "site": site,
                "timestamp": timestamp,
            }
        )


In [ ]:
surface_timestamps_df = pd.DataFrame.from_records(surface_timestamps_records)
surface_timestamps_df


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="timestamp",
    hue="site",
)


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="site",
    hue="timestamp",
)
